In [1]:
import spacy
import en_core_web_sm
import pandas as pd
import nltk
import re
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from keybert import KeyBERT
import json

In [2]:
#reading data
with open("C:\Lachin\DataSets\SMS\Text1.txt",'r') as file:
    details = file.readlines()

In [3]:
#Joining sentences
details = "".join(details)
details

"Apple Computers, Inc. was founded on April 1, 1976, by college dropouts Steve Jobs and Steve Wozniak, who brought to the new company a vision of changing the way people viewed computers. Jobs and Wozniak wanted to make computers small enough for people to have them in their homes or offices. Simply put, they wanted a computer that was user-friendly.\nJobs and Wozniak started out building the Apple I in Jobs' garage and sold them without a monitor, keyboard, or casing (which they decided to add on in 1977). The Apple II revolutionized the computer industry with the introduction of the first-ever color graphics. Sales jumped from $7.8 million in 1978 to $117 million in 1980, the year Apple went public"

In [4]:
#preprocessing function
def preprocessing(text):
    text = re.sub(r'\n',"",text)
    return text

#preprocessing
pre_details = preprocessing(details)
print(details)

Apple Computers, Inc. was founded on April 1, 1976, by college dropouts Steve Jobs and Steve Wozniak, who brought to the new company a vision of changing the way people viewed computers. Jobs and Wozniak wanted to make computers small enough for people to have them in their homes or offices. Simply put, they wanted a computer that was user-friendly.
Jobs and Wozniak started out building the Apple I in Jobs' garage and sold them without a monitor, keyboard, or casing (which they decided to add on in 1977). The Apple II revolutionized the computer industry with the introduction of the first-ever color graphics. Sales jumped from $7.8 million in 1978 to $117 million in 1980, the year Apple went public


### Summarizing

In [6]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [7]:
summarizor = pipeline('summarization', model = 'facebook/bart-large-cnn')

In [8]:
# z = summarizor(details,max_length = 100, min_length = 70)

In [9]:
# z[0]['summary_text']

In [10]:
len(tokenizer.encode(details))

161

In [11]:
tokenizer.model_max_length

1024

In [12]:
def summarizor(text, max_length,min_length):
    assert max_length < len(tokenizer.encode(text)), "Max length must be smaller than {} ".format(len(tokenizer.encode(text)))
    assert len(tokenizer.encode(text)) < tokenizer.model_max_length, "Text document must contain more less information"
    summary = summarizor(text,max_length=max_length, min_length=min_length)
    text = summary[0]['summary_text']
    return text

### Key word extraction

In [13]:
kw_model = KeyBERT()
# key_words = kw_model.extract_keywords(details,keyphrase_ngram_range=(2,2), top_n=3)
# key_words[0][0]

In [14]:
dict1 = {
    "SMS":{
        "Title":"Apple",
        "Details":"Manufacture phones"
    }
}

In [15]:
dict1

{'SMS': {'Title': 'Apple', 'Details': 'Manufacture phones'}}

In [18]:
outfile = open("C:\Lachin\DataSets\SMS\Text1.json",'w')
json.dump(dict1,outfile)

In [19]:
with open("C:\Lachin\DataSets\SMS\Text1.json",'r') as f:
    text = json.load(f)

In [20]:
text

{'SMS': {'Title': 'Apple', 'Details': 'Manufacture phones'}}

In [21]:
class SMS_summarizor(object):
    
    def __init__(self,paragraph):
        self.para = paragraph
        self.__para1 = self.__preprocessor(self.para)
        
    def __preprocessor(self,text):
        text = re.sub(r'\n',"",text)
        return text
    
    def summarizor(self, max_length,min_length):
        assert max_length < len(tokenizer.encode(self.__para1)), "Max length must be smaller than {} ".format(len(tokenizer.encode(self.__para1)))
        assert len(tokenizer.encode(self.__para1)) < tokenizer.model_max_length, "Text document must contain more less information"
        summary = summarizor(self.__para1,max_length=max_length, min_length=min_length)
        self.text = summary[0]['summary_text']
        
    def title_generator(self,number_of_words):
        assert type(number_of_words) == int,"number of words must be an integer"
        assert number_of_words <3, "Limit the words to 3"
        key_words = kw_model.extract(self.__para1,keyphrase_ngram_range=(number_of_words,number_of_words), top_n=3)
        self.title = key_words[0][0]
        
    def save_json(self,path,json_file_name):
        """
        Save the Json file of the summarized content and it's generated title.
        path ---> Path must be ended with \
                    ex-:C:\Lachin\DataSets\SMS\
        json_file_name ---> name.json
        """
        assert type(path) == str and type(json_file_name) == str , "Pathn and json_file_name must be string"
        assert json_file_name.endswith("json"),"It must be a .json extension"
        dict1 = {
            "SMS_Summarizor":{
                "Title": str(self.title),
                "Summary": str(self.text)
            }
        }
        
        location = open(path + json_file_name,'w')
        json.dump(dict1,location, indent = 2)
        print("Json File successfull saved.")

In [22]:
s1 = SMS_summarizor(details)

In [ ]:
s1.summarizor(100,70)